In [1]:
import bempp.api
import numpy as np

bempp.api.global_parameters.quadrature.medium.double_order = 4
bempp.api.global_parameters.quadrature.far.double_order = 4

In [2]:
import numpy
from scipy.sparse import coo_matrix, triu
import sys


class Mesh:
    """
    Store the verts and elements and physical data
    attributes
    ----------
    Verts : array
        array of 3d coordinates (npts x 3)
    Elmts : dict
        dictionary of tuples
        (rank 1 array of physical ids, rank 2 array of element to vertex ids
        (Nel x ppe)) each array in the tuple is of length nElmts Phys : dict
        keys and names
    methods
    -------
    read_msh:
        read a 2.0 ascii gmsh file
    write_neu:
        write a gambit neutral file. works for tets, tris in 3d and 2d
    write_vtu:
        write VTK file (calling vtk_writer.py)
    """
    def __init__(self):

        self.Verts = []
        self.Elmts = {}
        self.Phys = {}

        self.npts = 0
        self.nElmts = {}
        self.nprops = 0

        self._elm_types()   # sets elm_type

        self.meshname = ""

    def read_msh(self, mshfile):
        """Read a Gmsh .msh file.
        Reads Gmsh 2.0 mesh files
        """
        self.meshname = mshfile
        try:
            fid = open(mshfile, "r")
        except IOError:
            print("File '%s' not found." % mshfile)
            sys.exit()

        line = 'start'
        while line:
            line = fid.readline()

            if line.find('$MeshFormat') == 0:
                line = fid.readline()
                if line.split()[0][0] is not '2':
                    print("wrong gmsh version")
                    sys.exit()
                line = fid.readline()
                if line.find('$EndMeshFormat') != 0:
                    raise ValueError('expecting EndMeshFormat')

            if line.find('$PhysicalNames') == 0:
                line = fid.readline()
                self.nprops = int(line.split()[0])
                for i in range(0, self.nprops):
                    line = fid.readline()
                    newkey = int(line.split()[1])
                    qstart = line.find('"')+1
                    qend = line.find('"', -1, 0)-1
                    self.Phys[newkey] = line[qstart:qend]
                line = fid.readline()
                if line.find('$EndPhysicalNames') != 0:
                    raise ValueError('expecting EndPhysicalNames')

            if line.find('$ParametricNodes') == 0:
                line = fid.readline()
                self.npts = int(line.split()[0])
                self.Verts = numpy.zeros((self.npts, 3), dtype=float)
                for i in range(0, self.npts):
                    line = fid.readline()
                    data = line.split()
                    idx = int(data[0]) - 1  # fix gmsh 1-based indexing
                    if i != idx:
                        raise ValueError('problem with vertex ids')
                    self.Verts[idx, :] = list(map(float, data[1:4]))
                line = fid.readline()
                if line.find('$EndParametricNodes') != 0:
                    raise ValueError('expecting EndParametricNodes')

            if line.find('$Nodes') == 0:
                line = fid.readline()
                self.npts = int(line.split()[0])
                self.Verts = numpy.zeros((self.npts, 3), dtype=float)
                for i in range(0, self.npts):
                    line = fid.readline()
                    data = line.split()
                    idx = int(data[0]) - 1  # fix gmsh 1-based indexing
                    if i != idx:
                        raise ValueError('problem with vertex ids')
                    self.Verts[idx, :] = list(map(float, data[1:4]))
                line = fid.readline()
                if line.find('$EndNodes') != 0:
                    raise ValueError('expecting EndNodes')
                    

            if line.find('$Elements') == 0:
                line = fid.readline()
                self.nel = int(line.split()[0])
                for i in range(0, self.nel):
                    line = fid.readline()
                    data = line.split()
                    idx = int(data[0]) - 1  # fix gmsh 1-based indexing
                    if i != idx:
                        raise ValueError('problem with elements ids')
                    etype = int(data[1])           # element type
                    ntags = int(data[2])           # number of tags following
                    k = 3
                    if ntags > 0:                   # set physical id
                        physid = int(data[k+1])
                        if physid not in self.Phys:
                            self.Phys[physid] = 'Physical Entity %d' % physid
                            self.nprops += 1
                        k += ntags

                    verts = list(map(int, data[k:]))
                    verts = numpy.array(verts)  # fixe gmsh 1-based

                    if (etype not in self.Elmts) or\
                            (len(self.Elmts[etype]) == 0):
                        # initialize
                        self.Elmts[etype] = (physid, verts)
                        self.nElmts[etype] = 1
                    else:
                        # append
                        self.Elmts[etype] = \
                            (numpy.hstack((self.Elmts[etype][0], physid)),
                             numpy.vstack((self.Elmts[etype][1], verts)))
                        self.nElmts[etype] += 1
#                    print(self.Elmts)
    
                line = fid.readline()
                if line.find('$EndElements') != 0:
                    raise ValueError('expecting EndElements')
        fid.close()
    def _elm_types(self):
        elm_type = {}
        elm_type[1] = 2     # 2-node line
        elm_type[2] = 3     # 3-node triangle
        elm_type[3] = 4     # 4-node quadrangle
        elm_type[4] = 4     # 4-node tetrahedron
        elm_type[5] = 8     # 8-node hexahedron
        elm_type[6] = 6     # 6-node prism
        elm_type[7] = 5     # 5-node pyramid
        elm_type[8] = 3     # 3-node second order line
                            # (2 nodes at vertices and 1 with edge)
        elm_type[9] = 6     # 6-node second order triangle
                            # (3 nodes at vertices and 3 with edges)
        elm_type[10] = 9    # 9-node second order quadrangle
                            # (4 nodes at vertices,
                            #  4 with edges and 1 with face)
        elm_type[11] = 10   # 10-node second order tetrahedron
                            # (4 nodes at vertices and 6 with edges)
        elm_type[12] = 27   # 27-node second order hexahedron
                            # (8 nodes at vertices, 12 with edges,
                            #  6 with faces and 1 with volume)
        elm_type[13] = 18   # 18-node second order prism
                            # (6 nodes at vertices,
                            #  9 with edges and 3 with quadrangular faces)
        elm_type[14] = 14   # 14-node second order pyramid
                            # (5 nodes at vertices,
                            #  8 with edges and 1 with quadrangular face)
        elm_type[15] = 1    # 1-node point
        elm_type[16] = 8    # 8-node second order quadrangle
                            # (4 nodes at vertices and 4 with edges)
        elm_type[17] = 20   # 20-node second order hexahedron
                            # (8 nodes at vertices and 12 with edges)
        elm_type[18] = 15   # 15-node second order prism
                            # (6 nodes at vertices and 9 with edges)
        elm_type[19] = 13   # 13-node second order pyramid
                            # (5 nodes at vertices and 8 with edges)
        elm_type[20] = 9    # 9-node third order incomplete triangle
                            # (3 nodes at vertices, 6 with edges)
        elm_type[21] = 10   # 10-node third order triangle
                            # (3 nodes at vertices, 6 with edges, 1 with face)
        elm_type[22] = 12   # 12-node fourth order incomplete triangle
                            # (3 nodes at vertices, 9 with edges)
        elm_type[23] = 15   # 15-node fourth order triangle
                            # (3 nodes at vertices, 9 with edges, 3 with face)
        elm_type[24] = 15   # 15-node fifth order incomplete triangle
                            # (3 nodes at vertices, 12 with edges)
        elm_type[25] = 21   # 21-node fifth order complete triangle
                            # (3 nodes at vertices, 12 with edges, 6 with face)
        elm_type[26] = 4    # 4-node third order edge
                            # (2 nodes at vertices, 2 internal to edge)
        elm_type[27] = 5    # 5-node fourth order edge
                            # (2 nodes at vertices, 3 internal to edge)
        elm_type[28] = 6    # 6-node fifth order edge
                            # (2 nodes at vertices, 4 internal to edge)
        elm_type[29] = 20   # 20-node third order tetrahedron
                            # (4 nodes at vertices, 12 with edges,
                            #  4 with faces)
        elm_type[30] = 35   # 35-node fourth order tetrahedron
                            # (4 nodes at vertices, 18 with edges,
                            #  12 with faces, 1 in volume)
        elm_type[31] = 56   # 56-node fifth order tetrahedron
                            # (4 nodes at vertices, 24 with edges,
                            #  24 with faces, 4 in volume)
        self.elm_type = elm_type


In [3]:
file = 'cilindro_10x100mm';
    #boundary condition 0:temp 1:flux, not define is zero flux [1, 0] where [type, value]
# surf_bc={12:[0,0],5:[0,1]}
dirichlet_segments = [1,2]
mymesh = Mesh() #criou objeto: instanciou a classe
#chamei o metodo do objeto
mymesh.read_msh(file + '.msh')
XYZ = mymesh.Verts
# get only the triangular elements [[physid],[nodes]] physid is the surf of the elem
tri = mymesh.Elmts[2][1]-1
surf = mymesh.Elmts[2][0]-1

In [4]:
print(tri.shape)
print(surf.shape)
surf

(608, 3)
(608,)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0,

In [5]:
surf[50]

0

In [6]:
domain_index=surf
segments=np.unique(surf)
neumann_segments=[iseg for iseg in segments if iseg not in dirichlet_segments]
grid= bempp.api.grid_from_element_data(XYZ.transpose(), tri.transpose(),domain_index)

In [7]:
order_neumann = 0
order_dirichlet = 1

global_neumann_space = bempp.api.function_space(grid, "DP", order_neumann)
global_dirichlet_space = bempp.api.function_space(grid, "P", order_dirichlet)

neumann_space_dirichlet_segment = bempp.api.function_space(
    grid, "DP", order_neumann, domains=dirichlet_segments,
    closed=True, element_on_segment=True)

neumann_space_neumann_segment = bempp.api.function_space(
    grid, "DP", order_neumann, domains=neumann_segments,
    closed=False, element_on_segment=True, reference_point_on_segment=False)

dirichlet_space_dirichlet_segment = bempp.api.function_space(
    grid, "P", order_dirichlet, domains=dirichlet_segments, closed=True)

dirichlet_space_neumann_segment = bempp.api.function_space(
    grid, "P", order_dirichlet, domains=neumann_segments, closed=False)

dual_dirichlet_space = bempp.api.function_space(
    grid, "P", order_dirichlet, domains=dirichlet_segments,
    closed=True, strictly_on_segment=True)

In [8]:
slp_DD = bempp.api.operators.boundary.laplace.single_layer(
    neumann_space_dirichlet_segment,
    dirichlet_space_dirichlet_segment,
    neumann_space_dirichlet_segment)

dlp_DN = bempp.api.operators.boundary.laplace.double_layer(
    dirichlet_space_neumann_segment,
    dirichlet_space_dirichlet_segment,
    neumann_space_dirichlet_segment)

adlp_ND = bempp.api.operators.boundary.laplace.adjoint_double_layer(
    neumann_space_dirichlet_segment,
    neumann_space_neumann_segment,
    dirichlet_space_neumann_segment)

hyp_NN = bempp.api.operators.boundary.laplace.hypersingular(
    dirichlet_space_neumann_segment,
    neumann_space_neumann_segment,
    dirichlet_space_neumann_segment)

slp_DN = bempp.api.operators.boundary.laplace.single_layer(
    neumann_space_neumann_segment,
    dirichlet_space_dirichlet_segment,
    neumann_space_dirichlet_segment)

dlp_DD = bempp.api.operators.boundary.laplace.double_layer(
    dirichlet_space_dirichlet_segment,
    dirichlet_space_dirichlet_segment,
    neumann_space_dirichlet_segment)

id_DD = bempp.api.operators.boundary.sparse.identity(
    dirichlet_space_dirichlet_segment,
    dirichlet_space_dirichlet_segment,
    neumann_space_dirichlet_segment)

adlp_NN = bempp.api.operators.boundary.laplace.adjoint_double_layer(
    neumann_space_neumann_segment,
    neumann_space_neumann_segment,
    dirichlet_space_neumann_segment)

id_NN = bempp.api.operators.boundary.sparse.identity(
    neumann_space_neumann_segment,
    neumann_space_neumann_segment,
    dirichlet_space_neumann_segment)

hyp_ND = bempp.api.operators.boundary.laplace.hypersingular(
    dirichlet_space_dirichlet_segment,
    neumann_space_neumann_segment,
    dirichlet_space_neumann_segment)

blocked = bempp.api.BlockedOperator(2, 2)

blocked[0, 0] = slp_DD
blocked[0, 1] = -dlp_DN
blocked[1, 0] = adlp_ND
blocked[1, 1] = hyp_NN

In [9]:
def dirichlet_data_fun(x,domain_index):
    if(domain_index==1):
        return 1
    elif(domain_index==2):
        return 1
    else:
        return 0
    
def dirichlet_data(x, n, domain_index, res):
    res[0] = dirichlet_data_fun(x,domain_index)
    
def neumann_data_fun(x):
    return 0
 
def neumann_data(x, n, domain_index, res):
    res[0] = neumann_data_fun(x)

dirichlet_grid_fun = bempp.api.GridFunction(
    dirichlet_space_dirichlet_segment,
    fun=dirichlet_data, dual_space=dual_dirichlet_space)

neumann_grid_fun = bempp.api.GridFunction(
    neumann_space_neumann_segment,
    fun=neumann_data, dual_space=dirichlet_space_neumann_segment)

rhs_fun1 = (.5 * id_DD + dlp_DD) * dirichlet_grid_fun \
           - slp_DN * neumann_grid_fun
rhs_fun2 = - hyp_ND * dirichlet_grid_fun \
           + (.5 * id_NN - adlp_NN) * neumann_grid_fun

In [10]:
lhs = blocked.weak_form()
rhs = np.hstack([rhs_fun1.projections(neumann_space_dirichlet_segment), 
                 rhs_fun2.projections(dirichlet_space_neumann_segment)])
from scipy.sparse.linalg import gmres
x, info = gmres(lhs, rhs)

In [11]:
nx0 = neumann_space_dirichlet_segment.global_dof_count

neumann_solution = bempp.api.GridFunction(
    neumann_space_dirichlet_segment, coefficients=x[:nx0])
dirichlet_solution = bempp.api.GridFunction(
    dirichlet_space_neumann_segment, coefficients=x[nx0:])

In [12]:
bempp.api.PLOT_BACKEND = "ipython_notebook"

In [14]:
neumann_imbedding_dirichlet_segment = \
    bempp.api.operators.boundary.sparse.identity(
        neumann_space_dirichlet_segment,
        global_neumann_space,
        global_neumann_space)

neumann_imbedding_neumann_segment = \
    bempp.api.operators.boundary.sparse.identity(
        neumann_space_neumann_segment,
        global_neumann_space,
        global_neumann_space)

dirichlet_imbedding_dirichlet_segment = \
    bempp.api.operators.boundary.sparse.identity(
        dirichlet_space_dirichlet_segment,
        global_dirichlet_space,
        global_dirichlet_space)

dirichlet_imbedding_neumann_segment = \
    bempp.api.operators.boundary.sparse.identity(
        dirichlet_space_neumann_segment,
        global_dirichlet_space,
        global_dirichlet_space)

dirichlet = (dirichlet_imbedding_dirichlet_segment * dirichlet_grid_fun +
             dirichlet_imbedding_neumann_segment * dirichlet_solution)

neumann = (neumann_imbedding_neumann_segment * neumann_grid_fun +
           neumann_imbedding_dirichlet_segment * neumann_solution)

dirichlet.plot()

A Jupyter Widget

A Jupyter Widget

In [15]:
# Print out the number of elements
number_of_elements = grid.leaf_view.entity_count(0)
print("The grid has {0} elements.".format(number_of_elements))

The grid has 608 elements.


In [16]:
number_of_global_neumann_dofs = global_neumann_space.global_dof_count
print("Number of global neumann dofs: {0}".format(number_of_global_neumann_dofs))

Number of global neumann dofs: 608


In [17]:
number_of_global_dirichlet_dofs = global_dirichlet_space.global_dof_count
print("Number of global dirichlet dofs: {0}".format(number_of_global_dirichlet_dofs))

Number of global dirichlet dofs: 306


In [18]:
print("Number of global dofs: {0}".format(number_of_global_dirichlet_dofs+number_of_global_neumann_dofs))

Number of global dofs: 914
